# **VGG16**

In [1]:
import numpy as np
from glob import glob
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
IMG_ROWS = 224      # for vgg16
IMG_COLS = 224      # for vgg16
IMG_CHANNELS = 3
CLASSES = 2         # mountain and sea

In [5]:
# Load images and label the images
def load_images_and_labels(files, label):
    images = [image.load_img(p, target_size=(IMG_ROWS, IMG_COLS)) for p in files]
    labels = [label] * len(files)
    return images, labels

mountain_files = glob('/content/drive/MyDrive/Samples/crawling/m_mountain/*jpg')
sea_files = glob('/content/drive/MyDrive/Samples/crawling/s_sea/*jpg')

mountain_images, mountain_labels = load_images_and_labels(mountain_files, 0)    # label mountain as 0
sea_images, sea_labels = load_images_and_labels(sea_files, 1)                   # label sea as 1

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (109542160 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [6]:
# Concatenate mountain data and sea data
images = mountain_images + sea_images
labels = mountain_labels + sea_labels

In [7]:
# Vectorize the images
vector = np.asarray([image.img_to_array(img) for img in images])
print(vector.shape)

(374, 224, 224, 3)


In [8]:
# Load VGG16 model
base_model = VGG16(weights='imagenet', include_top=True)
base_model.summary()

553467096/553467096 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [10]:
# Modifiy base model
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
# Use model's output as ML model's input
vectors = model.predict(preprocess_input(vector))
print(vectors.shape)

12/12 [==============================] - 206s 17s/step
(374, 4096)


# **ML**

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import xgboost as xgb

In [13]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(vectors, labels, test_size=0.2)

### **Logistic Regression**

In [14]:
# Initialize logistic regression model
lr_1 = LogisticRegression(C=0.1, penalty='l1', solver='saga', max_iter=10000)
# C: 정규화 강도의 역수 (C값이 클수록 정규화 강도는 작아지고 과적합 위험이 커짐)

# Fit the model
lr_1.fit(X_train, y_train)

# Evaluate the model
score = lr_1.score(X_test, y_test)
print(f'Score: {score:.4f}')

Score: 0.9200


In [15]:
# Initialize logistic regression model
lr_2 = LogisticRegression(C=10, penalty='l2', solver='saga', max_iter=500)

# Fit the model
lr_2.fit(X_train, y_train)

# Evaluate the model
score = lr_2.score(X_test, y_test)
print(f'Score: {score:.4f}')

Score: 0.9067


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


### **SVM**

In [16]:
# Initialize svm model
svm_model = svm.SVC(kernel='rbf')

# Fit the model
svm_model.fit(X_train, y_train)

# Evaluate the model
score = svm_model.score(X_test, y_test)
print(f'Score: {score:.4f}')

Score: 0.9067


### **XGB**

In [17]:
# Initialize xgb model
xgb_model = xgb.XGBClassifier(max_depth=2, n_estimators=1000)

# Fit the model
xgb_model.fit(X_train, y_train)

# Evaluate the model
score = xgb_model.score(X_test, y_test)
print(f'Score: {score:.4f}')

Score: 0.9067
